In [2]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt
from IPython.display import HTML

In [3]:
IMAGE_SIZE = 256
CHANNELS = 3

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=10,
)
train_generator = train_datagen.flow_from_directory(
        'spdataset/train',
        target_size=(IMAGE_SIZE,IMAGE_SIZE),
        batch_size=32,
        class_mode="sparse",
    
)

Found 2307 images belonging to 2 classes.


In [4]:
train_generator.class_indices

{'benign': 0, 'malignant': 1}

In [5]:
class_names = list(train_generator.class_indices.keys())
class_names

['benign', 'malignant']

In [6]:
validation_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=10,
        horizontal_flip=True)
validation_generator = validation_datagen.flow_from_directory(
        'spdataset/val',
        target_size=(IMAGE_SIZE,IMAGE_SIZE),
        batch_size=32,
        class_mode="sparse"
)

Found 329 images belonging to 2 classes.


In [7]:
test_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=10,
        horizontal_flip=True)

test_generator = test_datagen.flow_from_directory(
        'spdataset/test',
        target_size=(IMAGE_SIZE,IMAGE_SIZE),
        batch_size=32,
        class_mode="sparse"
)

Found 661 images belonging to 2 classes.


In [8]:
for image_batch, label_batch in test_generator:
    print(image_batch[0])
    break

[[[0.8313726  0.52156866 0.6039216 ]
  [0.8313726  0.52156866 0.6039216 ]
  [0.8313726  0.52156866 0.6039216 ]
  ...
  [0.855915   0.46357006 0.60082495]
  [0.8431785  0.46278638 0.6000413 ]
  [0.82745105 0.44705886 0.58431375]]

 [[0.8313726  0.5163261  0.6004266 ]
  [0.8313726  0.5180015  0.6015435 ]
  [0.8313726  0.5196768  0.6026604 ]
  ...
  [0.85814875 0.46971294 0.60696787]
  [0.8381525  0.45776036 0.5950153 ]
  [0.82745105 0.44705886 0.58431375]]

 [[0.82625    0.5029738  0.59607846]
  [0.8279253  0.5052076  0.59607846]
  [0.8296007  0.50744134 0.59607846]
  ...
  [0.86038256 0.47585583 0.6131107 ]
  [0.8331266  0.45273438 0.5899893 ]
  [0.82745105 0.44705886 0.58431375]]

 ...

 [[0.7960785  0.50980395 0.5882353 ]
  [0.7954478  0.50602025 0.5882353 ]
  [0.788613   0.4797774  0.5817382 ]
  ...
  [0.7535319  0.43588477 0.51039463]
  [0.7540903  0.43644324 0.510953  ]
  [0.75464875 0.4370017  0.5115115 ]]

 [[0.7960785  0.50980395 0.5882353 ]
  [0.79488945 0.5026696  0.5882353 ]


In [10]:
# input_shape = (IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
# n_classes = 3

# model = models.Sequential([
#     layers.InputLayer(input_shape=input_shape),
#     layers.Conv2D(32, kernel_size = (3,3), activation='relu'),
#     layers.MaxPooling2D((2, 2)),
#     layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
#     layers.MaxPooling2D((2, 2)),
#     layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
#     layers.MaxPooling2D((2, 2)),
#     layers.Conv2D(64, (3, 3), activation='relu'),
#     layers.MaxPooling2D((2, 2)),
#     layers.Conv2D(64, (3, 3), activation='relu'),
#     layers.MaxPooling2D((2, 2)),
#     layers.Conv2D(64, (3, 3), activation='relu'),
#     layers.MaxPooling2D((2, 2)),
#     layers.Flatten(),
#     layers.Dense(64, activation='relu'),
#     layers.Dense(n_classes, activation='softmax'),
# ])

In [11]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define constants
IMAGE_SIZE = 256  # Adjust this to match your dataset's image size
NUM_CLASSES = 2  # Number of classes in your dataset
BATCH_SIZE = 32
EPOCHS = 20

# Load the pre-trained ResNet-50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3))

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)  # You can adjust the number of units as needed
predictions = Dense(2, activation='softmax')(x)

# Create the fine-tuned model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
# Compile the model with 'sparse_categorical_crossentropy' as the loss function
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])





# # Save the model
# model.save('fine_tuned_resnet50.h5')

# Evaluate the model on a test dataset if available
# test_datagen = ImageDataGenerator(rescale=1.0 / 255)
# test_generator = test_datagen.flow_from_directory(
#     'test_data_directory',
#     target_size=(IMAGE_SIZE, IMAGE_SIZE),
#     batch_size=BATCH_SIZE,
#     class_mode='categorical'
# )
# test_loss, test_accuracy = model.evaluate(test_generator)
# print(f'Test loss: {test_loss}, Test accuracy: {test_accuracy}')


In [12]:
# Train the model
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=30,
    validation_data=validation_generator,
    validation_steps= validation_generator.samples // BATCH_SIZE,
    callbacks=[early_stopping]
)

Epoch 1/30
72/72 [==============================] - 369s 5s/step - loss: 0.9315 - accuracy: 0.5371 - val_loss: 0.7106 - val_accuracy: 0.5500
Epoch 2/30
72/72 [==============================] - 357s 5s/step - loss: 0.6999 - accuracy: 0.5468 - val_loss: 0.7867 - val_accuracy: 0.4531
Epoch 3/30
72/72 [==============================] - 357s 5s/step - loss: 0.6776 - accuracy: 0.5877 - val_loss: 0.6668 - val_accuracy: 0.5594
Epoch 4/30
72/72 [==============================] - 360s 5s/step - loss: 0.6970 - accuracy: 0.5495 - val_loss: 0.7775 - val_accuracy: 0.4531
Epoch 5/30
72/72 [==============================] - 349s 5s/step - loss: 0.6763 - accuracy: 0.5934 - val_loss: 0.7919 - val_accuracy: 0.4531
Epoch 6/30
72/72 [==============================] - 352s 5s/step - loss: 0.6847 - accuracy: 0.5798 - val_loss: 0.6249 - val_accuracy: 0.7688
Epoch 7/30
72/72 [==============================] - 389s 5s/step - loss: 0.6345 - accuracy: 0.6426 - val_loss: 0.5965 - val_accuracy: 0.7281
Epoch 8/30
72

In [13]:
# import tensorflow as tf
# from tensorflow.keras.applications import ResNet50
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
# from tensorflow.keras.preprocessing.image import ImageDataGenerator

# # Define constants
# IMAGE_SIZE = 256  # Adjust this to match your dataset's image size
# NUM_CLASSES = 2  # Number of classes in your dataset
# BATCH_SIZE = 32
# EPOCHS = 20

# # Load the pre-trained ResNet-50 model
# base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3))

# # Freeze the layers of the base model
# for layer in base_model.layers:
#     layer.trainable = False

# # Add custom classification layers on top of the base model
# x = base_model.output
# x = GlobalAveragePooling2D()(x)
# x = Dense(1024, activation='relu')(x)  # You can adjust the number of units as needed
# predictions = Dense(NUM_CLASSES, activation='softmax')(x)

# # Create the fine-tuned model
# model = Model(inputs=base_model.input, outputs=predictions)

# # Compile the model
# # Compile the model with 'sparse_categorical_crossentropy' as the loss function
# model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# # Create data generators for training and validation
# train_datagen = ImageDataGenerator(
#     rescale=1.0 / 255,
#     rotation_range=20,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True,
#     fill_mode='nearest'
# )

# # Assuming you have a directory structure with train and validation subdirectories
# train_generator = train_datagen.flow_from_directory(
#     'spdataset/train',
#     target_size=(IMAGE_SIZE, IMAGE_SIZE),
#     batch_size=BATCH_SIZE,
#     class_mode='sparse'  # Use 'sparse' for a small number of classes
# )

# # Validation data generator
# validation_datagen = ImageDataGenerator(rescale=1.0 / 255)
# validation_generator = validation_datagen.flow_from_directory(
#     'spdataset/val',
#     target_size=(IMAGE_SIZE, IMAGE_SIZE),
#     batch_size=BATCH_SIZE,
#     class_mode='sparse'  # Use 'sparse' for a small number of classes
# )

# # Train the model
# history = model.fit(
#     train_generator,
#     steps_per_epoch=len(train_generator),
#     epochs=10,
#     validation_data=validation_generator,
#     validation_steps=len(validation_generator)
# )

# # # Save the model
# # model.save('fine_tuned_resnet50.h5')


In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 127, 127, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 62, 62, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 60, 60, 64)        36928     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 30, 30, 64)        0

In [15]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [17]:
# Train the model
from tensorflow.keras.callbacks import EarlyStopping
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // 32,
    epochs=10,
    validation_data=validation_generator,
    validation_steps= validation_generator.samples // 32,
)

Epoch 1/10
72/72 [==============================] - 204s 3s/step - loss: 0.6931 - accuracy: 0.5499 - val_loss: 0.5323 - val_accuracy: 0.7312
Epoch 2/10
72/72 [==============================] - 180s 2s/step - loss: 0.5138 - accuracy: 0.7556 - val_loss: 0.4805 - val_accuracy: 0.7937
Epoch 3/10
72/72 [==============================] - 175s 2s/step - loss: 0.4894 - accuracy: 0.7657 - val_loss: 0.4375 - val_accuracy: 0.7969
Epoch 4/10
72/72 [==============================] - 181s 3s/step - loss: 0.4516 - accuracy: 0.7785 - val_loss: 0.4016 - val_accuracy: 0.8281
Epoch 5/10
72/72 [==============================] - 186s 3s/step - loss: 0.4368 - accuracy: 0.7833 - val_loss: 0.3788 - val_accuracy: 0.8438
Epoch 6/10
72/72 [==============================] - 177s 2s/step - loss: 0.4331 - accuracy: 0.7815 - val_loss: 0.4069 - val_accuracy: 0.8125
Epoch 7/10
72/72 [==============================] - 176s 2s/step - loss: 0.4240 - accuracy: 0.7956 - val_loss: 0.3897 - val_accuracy: 0.8344
Epoch 8/10
72

In [13]:
import os
model_version=max([int(i) for i in os.listdir("../models") + [0]])+1
model.save(f"../models/{model_version}")

INFO:tensorflow:Assets written to: ../models/3\assets


INFO:tensorflow:Assets written to: ../models/3\assets


In [1]:
model.save("../potatoesv2.h5")

NameError: name 'model' is not defined

NameError: name 'test_ds' is not defined